In [1]:
%matplotlib widget

import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.colors as mcolors
from matplotlib.patches import Rectangle

In [3]:
file_path = 'seg_length_km_GAIA_2019.nc'
dsl = xr.open_dataset(file_path)
print(dsl)

# Somme sur toutes les latitudes et longitudes pour chaque variable souhaitée
sum_seg_length_km = dsl['seg_length_km'].sum(dim=['lat', 'lon']).item()
sum_seg_length_contrail_km = dsl['seg_length_contrail_km'].sum(dim=['lat', 'lon']).item()

print(f"Somme seg_length_km: {sum_seg_length_km}")
print(f"Somme seg_length_contrail_km: {sum_seg_length_contrail_km}")


<xarray.Dataset> Size: 1MB
Dimensions:                 (lat: 180, lon: 360)
Coordinates:
  * lat                     (lat) float64 1kB -89.5 -88.5 -87.5 ... 88.5 89.5
  * lon                     (lon) float64 3kB 0.5 1.5 2.5 ... 357.5 358.5 359.5
Data variables:
    seg_length_km           (lat, lon) float64 518kB ...
    seg_length_contrail_km  (lat, lon) float64 518kB ...
Somme seg_length_km: 60929593892.37307
Somme seg_length_contrail_km: 12505560816.768536


In [4]:
file_path = 'EF_and_RF_per_contrail_length_2010_2019.nc'
ds = xr.open_dataset(file_path)
print(ds)

rf_lin_cleaned = ds['RF_net'].where(np.isfinite(ds['RF_lin']), np.nan)/10**6/10**9*5
mean_rf_lin = rf_lin_cleaned.mean().item()
n_lat = ds.sizes['lat']
n_lon = ds.sizes['lon']
ds['RFD'] = (('year', 'lat', 'lon'), np.full_like(ds['RF_net'], np.nan))  # initialisation (si nécessaire)
ds['RFDc'] = (('year', 'lat', 'lon'), np.full_like(ds['RF_net'], np.nan))  # initialisation (si nécessaire)

for k in range(2010, 2020):
    rf_net_total = ds['RF_net'].sel(year=k).sum(dim=['lat', 'lon']).item()/n_lat/n_lon*1000
    print(rf_net_total)
    new_var_data = ds.sel(year=k)['RF_net'] / dsl['seg_length_km']
    ds['RFD'].loc[dict(year=k)] = new_var_data
    new_var_data2 = ds.sel(year=k)['RF_net'] / dsl['seg_length_contrail_km']
    ds['RFDc'].loc[dict(year=k)] = new_var_data2

<xarray.Dataset> Size: 29MB
Dimensions:       (year: 10, lat: 180, lon: 360)
Coordinates:
  * year          (year) int64 80B 2010 2011 2012 2013 ... 2016 2017 2018 2019
  * lat           (lat) float64 1kB -89.5 -88.5 -87.5 -86.5 ... 87.5 88.5 89.5
  * lon           (lon) float64 3kB 0.5 1.5 2.5 3.5 ... 356.5 357.5 358.5 359.5
Data variables:
    RF_net        (year, lat, lon) float32 3MB ...
    areacella     (lat, lon) float64 518kB ...
    EF            (year, lat, lon) float64 5MB ...
    RF_lin_cotra  (year, lat, lon) float64 5MB ...
    RF_lin        (year, lat, lon) float64 5MB ...
    EF_lin_cotra  (year, lat, lon) float64 5MB ...
    EF_lin        (year, lat, lon) float64 5MB ...
Attributes:
    regrid_method:  conservative
39.30707013165509
37.43068742163387
39.6030718014564
39.86480336130401
40.32822220413773
41.69754216700425
40.26477201485339
41.52759316526814
39.94550163363233
41.882557810088734


In [6]:
# Charger le fichier NetCDF original
file_path = 'EF_and_RF_per_contrail_length_2010_2019.nc'
ds = xr.open_dataset(file_path)  # <-- mets ici le nom de ton fichier

# Calcul de la moyenne temporelle pour chaque variable dépendant de "year"
mean_vars = {}
for var in ds.data_vars:
    if "year" in ds[var].dims:
        mean_vars[var] = ds[var].mean(dim="year")
    else:
        mean_vars[var] = ds[var]  # garder tel quel si pas de dimension year

# Création du nouveau dataset avec les moyennes
ds_mean = xr.Dataset(mean_vars, coords={"lat": ds.lat, "lon": ds.lon})

# Sauvegarde en NetCDF
ds_mean.to_netcdf("output_mean.nc")
print(ds_mean)
print("Fichier créé : output_mean.nc")

<xarray.Dataset> Size: 3MB
Dimensions:       (lat: 180, lon: 360)
Coordinates:
  * lat           (lat) float64 1kB -89.5 -88.5 -87.5 -86.5 ... 87.5 88.5 89.5
  * lon           (lon) float64 3kB 0.5 1.5 2.5 3.5 ... 356.5 357.5 358.5 359.5
Data variables:
    RF_net        (lat, lon) float32 259kB 0.0 0.0 0.0 ... 0.007465 0.007465
    areacella     (lat, lon) float64 518kB ...
    EF            (lat, lon) float64 518kB 0.0 0.0 0.0 ... 1.064e+14 1.064e+14
    RF_lin_cotra  (lat, lon) float64 518kB nan nan nan ... 2.765e-05 3.414e-05
    RF_lin        (lat, lon) float64 518kB nan nan nan ... 5.748e-06 7.097e-06
    EF_lin_cotra  (lat, lon) float64 518kB nan nan nan ... 3.942e+08 4.866e+08
    EF_lin        (lat, lon) float64 518kB nan nan nan ... 8.193e+07 1.012e+08
Fichier créé : output_mean.nc


C:\Users\t.planes\AppData\Local\miniconda3\envs\AeroMetrics\lib\site-packages\numpy\_core\fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
C:\Users\t.planes\AppData\Local\miniconda3\envs\AeroMetrics\lib\site-packages\numpy\_core\fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
